In [104]:
%matplotlib inline

In [71]:
path = "../../data/dl1/obj_neural_code/"
#path = "../../data/obj_neural_code/sample/"
modelpath = "../../models/dl1/vgg16_ft_obj_neural_code/"

In [18]:
pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [27]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [28]:
import utils; reload(utils)
from utils import plots

In [29]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

In [5]:
from keras.models import Model
from keras.models import model_from_json

In [20]:
batch_size = 64
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

Found 1120 images belonging to 40 classes.
Found 160 images belonging to 40 classes.


In [7]:
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=10)

Epoch 1/10
1120/1120 [==============================] - 33s - loss: 3.7648 - acc: 0.2036 - val_loss: 1.0013 - val_acc: 0.7250
Epoch 2/10
1120/1120 [==============================] - 32s - loss: 1.3866 - acc: 0.6018 - val_loss: 0.6312 - val_acc: 0.7750
Epoch 3/10
1120/1120 [==============================] - 33s - loss: 0.9201 - acc: 0.7277 - val_loss: 0.4510 - val_acc: 0.8500
Epoch 4/10
1120/1120 [==============================] - 33s - loss: 0.6569 - acc: 0.7911 - val_loss: 0.4159 - val_acc: 0.8500
Epoch 5/10
1120/1120 [==============================] - 33s - loss: 0.5685 - acc: 0.8321 - val_loss: 0.3733 - val_acc: 0.8938
Epoch 6/10
1120/1120 [==============================] - 33s - loss: 0.5020 - acc: 0.8277 - val_loss: 0.2975 - val_acc: 0.8812
Epoch 7/10
1120/1120 [==============================] - 33s - loss: 0.4352 - acc: 0.8750 - val_loss: 0.3921 - val_acc: 0.8750
Epoch 8/10
1120/1120 [==============================] - 33s - loss: 0.3838 - acc: 0.8750 - val_loss: 0.2624 - val_acc:

In [9]:
base = vgg.model
model_json = base.to_json()
with open(modelpath + "base.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
base.save_weights(modelpath+"base.h5")

In [10]:
idx = [0,5,10,17,24,31,33,35,37]
layers  = []
for i in range(len(idx)):
    print(base.layers[idx[i]].name)
    layers.append( base.layers[idx[i]] )

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_4


In [17]:
models = []
for layer in layers:
    #print base_model.get_layer(layer.name)
    models.append( Model(input=base.input,
                         output=base.get_layer(layer.name).output) )
    
for model in models:
    NAME = model.layers[-1].name    
    model_json = model.to_json()    
    with open(modelpath + NAME + '.json', 'w') as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(modelpath + NAME + '.h5')
print("Saved models to disk")

Saved models to disk


In [72]:
filelist = []
range(1,40 + 1)
for cat in range(1,40+1):
    for f in os.listdir(path + 'train/' + str(cat)):
        if f.endswith('.png'):
            filename = path + 'train/' + str(cat) + '/' + f
            filelist.append(filename)                
print('Filelist with ' + str(len(filelist)) + ' images')

Filelist with 1120 images


In [117]:
newpath = "../../data/dl1/obj_neural_code/all_flat/"
batch_size = 60
batches = vgg.get_batches(newpath, batch_size=batch_size, shuffle=False)

Found 1440 images belonging to 1 classes.


In [119]:
NAME = models[5].layers[-1].name  
NAME

nbatches = int(1440/batch_size)
nbatches

24

In [122]:
from time import time
R = []
for n in range(nbatches):
    imgs,_ = batches.next()
    ti = time()
    R.append( models[5].predict(imgs) )
    te = time() - ti
    print(te) 

1.53175282478
1.50188279152
1.46206498146
1.48755502701
1.4870531559
1.48069310188
1.48744010925
1.49954295158
1.48403978348
1.50717401505
1.50781297684
1.48199987411
1.49620914459
1.50108718872
1.50102806091
1.48641014099
1.49899291992
1.52116799355
1.5209209919
1.50491905212
1.48009395599
1.51470303535
1.5037779808
1.48276209831


In [126]:
R2 = np.asarray(R, dtype=float)

In [128]:
R2.shape = [1440,512,7,7]

In [129]:
R2.shape

(1440, 512, 7, 7)